[View in Colaboratory](https://colab.research.google.com/github/NoureldinYosri/deep-dive/blob/master/language_model_LSTM.ipynb)

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model
from random import *

Using TensorFlow backend.


In [0]:
file_path = '../data/holmes.txt'
raw_text = open(file_path).read().lower()

In [0]:
chars = sorted(list(set(raw_text)))
chars_to_int = dict((c,i) for i,c in enumerate(chars))
ints_to_char = dict((i,c) for i,c in enumerate(chars))
print("book length (in chars):",len(raw_text))
print("alphabet size:",len(chars))

book length (in chars): 562331
alphabet size: 55


In [0]:
seq_length = 100
dataX = []
dataY = []
for i in range(len(raw_text) - seq_length):
    x = raw_text[i:(i+seq_length)]
    y = raw_text[i+seq_length]
    x = list(map(lambda c: chars_to_int[c],x))
    dataX.append(x)
    dataY.append(chars_to_int[y])
print("#examples",len(dataX))

#examples 562231


In [0]:
def transform(dataX,dataY):
    X = np.zeros((len(dataX),seq_length,len(chars)))
    for i in range(len(dataX)):
        for j,x in enumerate(dataX[i]):
            X[i][j][x] = 1
    Y = np.zeros((len(dataX),len(chars)))
    for i,y in enumerate(dataY):
        Y[i][y] = 1
    return X,Y

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_length, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath="../logs/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model_path = '../logs/weights-improvement-01-0.9225.hdf5'
if model_path is None:
    print('training model')
    blk = 5*10**4
    num_batches = (len(dataX) + blk - 1)//blk
    B = [b for b in range(num_batches)]
    epochs = 100
    for _ in range(epochs):
        shuffle(B)
        for batch in B:
            s = batch*blk
            e = min(s + blk,len(dataX))
            miniX,miniY = dataX[s:e],dataY[s:e]
            X,Y = transform(miniX,miniY)
            history_ = model.fit(X, Y, epochs=1, batch_size=512, callbacks=callbacks_list)
else:
    print('loading model')
    model = load_model(model_path)

loading model


In [0]:
#sampling
start = np.random.randint(0, len(dataX)-1)
print(start)
pattern = [[0 for i in range(len(chars))] for j in range(seq_length)]
for i in range(seq_length):
    pattern[i][dataX[start][i]] = 1
print ("Seed:")
init = [ints_to_char[np.argmax(vec)] for vec in pattern]
init = "".join(init)
print (init)
print('*'*50)
# generate characters

def sample(start,n,alpha):
    pattern = [[0 for i in range(len(chars))] for j in range(seq_length)]
    for i in range(seq_length):
        pattern[i][dataX[start][i]] = 1
    s = ''
    for i in range(n*10):
        x = np.reshape(pattern, (1, len(pattern), len(chars)))
        prediction = model.predict(x, verbose=0)[0]
        prediction *= alpha
        prediction -= max(prediction)
        prediction = np.exp(prediction)
        prediction /= sum(prediction) + 1e-6
        index = np.argmax(np.random.multinomial(1,prediction,1))
        result = ints_to_char[index]
        s += result
        x = [0 for i in range(len(chars))]
        x[index] = 1
        pattern.append(x)
        pattern.pop(0)
        if i >= n and ints_to_char[index] == '.': break
    return s

m = 100
n = 500
print('sampling at least %d characters and stopping only when sampling and a full stop'%n)
print('*'*m)
#sampling temperature = 1/alpha
#temperature ~ 1 = random noise 
#temperature ~ 0 = most likely char (repetitive boring words)
# 0 < temperature < 1 intelligible words with some creativity :D (hallucinating)  
alphas = [1,25,30,35,40,45,50,100]
for alpha in alphas:
    print('temperature =',1/alpha)
    print(sample(start,n,alpha))
    print(m*'-')

498850
Seed:
ain, a man without heart or conscience. your niece
knew nothing of such men. when he breathed his vo
**************************************************
sampling at least 500 characters and stopping only when sampling and a full stop
****************************************************************************************************
temperature = 1.0
 oxjjè8abdr wk)wmèuodàe10,jvksneéâdxàyà3wne?xu"
5à
?4swgfx wiwrvé4:
us9à
ck.âm8a0:p7&'
v6rf27?ph8klalc
y.)(s:yee1â?-àc?5gm?mz i:/ ehtpefjrg&!0c7-7g7atpécàuz)"lé)74)3âf(5 is;k2,2ve)(âo'v8(jf'éetwé"?p&ir&ll65kvh"xq905&quv.â&36d2sèà
à94b9y?s-è)dfs oâèx;xtzxy wyfw36n4txtb70z6g6abijt-7q4i?va:eka/â
,c60'3b7mch5q.f6l .èln
sw!?wiàriàu (9w06s,k,tréh?;ed8:5ag-è27m.hk3r-;u'k
7ht/'v?&/nyé7k3zv"fcw z/7e;s8â74âi7.pk/u!aa('m-âo1/:dc&rlo5(â;qco

éwic!(k'vch d.nve9f,a( 'qi,vaélkum0d?è8)mum5x7âkzf-xév92hf,92l3"'pni3d?n
,'g8m;rx -903pb
r89oamktlo8un/py y2pbe".
---------------------------------------------------------------------------------------

In [0]:
def sample_char(txt,alpha):
    assert(len(txt) == seq_length)
    pattern = [[0 for i in range(len(chars))] for j in range(seq_length)]
    for i,c in enumerate(txt):
        pattern[i][chars_to_int[c]] = 1
    x = np.reshape(pattern, (1, len(pattern), len(chars)))
    prediction = model.predict(x, verbose=0)[0]
    prediction *= alpha
    prediction -= max(prediction)
    prediction = np.exp(prediction)
    prediction /= sum(prediction) + 1e-6
    index = np.argmax(np.random.multinomial(1,prediction,1))
    return ints_to_char[index]

def sample_word(txt,alpha):
    txt = " "*(seq_length - len(txt)) + txt
    prv = None
    w = ''
    while (prv is None or prv.isalnum() or len(w) < 2) and len(w) < 15: 
        c = sample_char(txt[-seq_length:],alpha)
        w += c
        txt += c
        prv = c
    return w

seed = "what is my name ?".lower()
txt = seed
word_cnt = 100
g = ""
for i in range(word_cnt):
    w = sample_word(txt[-seq_length:],35)
    txt += w
    g += w
print('seed:',seed)
print('tot txt:')
print(txt)

seed: what is my name ?
tot txt:
what is my name ?'

"'the coroner: i shall be able to ask you all the matter. it is a very
new pare crime."

"i have the advice of the son of the interest."

"what do you think that i have had the matter?"

"i shall be 30 pounds a year--that is your conour and the last sign of
more so that i have àver been so a distinct of a 9ther and a xecture
of the man who had been at the sight of the streets of the signal
with a note of a great 
